In [ ]:
import numpy as np
import json
import glob
from functools import reduce

In [ ]:
def load_results(dataset, model, metric, n_sample, seed):
    result_dir = f'../results/{dataset}/{model}_{seed}/{metric}_{n_sample}'
      
    result_fs = glob.glob(f'{result_dir}/*.json')
    results = []
    for file in result_fs:
        with open(file, 'r') as f:
            data = json.load(f)
            print(file, len(data))
        results.extend(data)

    return results

In [ ]:
metric = 'wd'
dataset = 'CIFAR-10'
arch = 'resnet18'
n_sample = 10000
seed = '1_15'
logits_results = load_results(dataset, arch, metric, n_sample, seed)

In [ ]:
len(logits_results)

In [ ]:
import math

In [ ]:
import altair as alt
import pandas as pd

In [ ]:
alt.Chart(alt.Data(values=logits_results)).mark_point().encode(
    x=alt.X('metric:Q', title=metric),
    y='error:Q',
    color=alt.Color('corruption:N', scale=alt.Scale(scheme='category20')),
    shape='corruption level:O'
).properties(
    width=250,
    height=200
)

In [ ]:
def polyfit(x, y, degree=1):
    results = {}

    coeffs = np.polyfit(x, y, degree)

    results['polynomial'] = coeffs.tolist()

    p = np.poly1d(coeffs)

    yhat = p(x)                
    ybar = np.sum(y)/len(y)          
    ssreg = np.sum((yhat - ybar)**2)   
    sstot = np.sum((y - ybar)**2)    
    results['determination'] = ssreg / sstot

    return results

In [ ]:
from scipy.stats import spearmanr
import math

In [ ]:
d = [i['metric'] for i in logits_results if not math.isnan(i['metric'])]
e = [i['error'] for i in logits_results if not math.isnan(i['metric'])]
print(f'{metric} r2:', polyfit(d, e)['determination'])

coef, p = spearmanr(d, e)
print(f'{metric} rho:', coef)

In [ ]:
print('slope:', polyfit(d, e)['polynomial'][0])
print('bias:', polyfit(d, e)['polynomial'][1])